In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer, AutoModel
print(torch.cuda.is_available())

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [ ]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small", trust_remote_code=True)

In [ ]:
repo_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(repo_name, trust_remote_code=True)
model = model.cuda().to(torch.float16)

In [5]:

# Chat with Hymba
prompt = "Plate: WATDFCK\nMeaning: "
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=64, do_sample=False, temperature=0.7, use_cache=True)
response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

print(f"Model response: {response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Model response: 1. The word "WATDFCK" is a combination of the words "Watermelon" and "Dick." It is a playful way to refer to a male friend or acquaintance. 2. The word "


In [ ]:
import json

ca_plates_df = pd.read_csv("../combined_clustered_plates_with_classified_majority_vote.csv")

jsonl_data = []
for _, row in ca_plates_df.iterrows():
    jsonl_data.append({
        "prompt": f"Given the following Plate: {row['plate']}\nMeaning:",
        "completion": f" {row['customer_meaning']}"
    })

with open("formatted_dataset.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="formatted_dataset.jsonl")

def tokenize_function(examples):
    return tokenizer(
        examples["prompt"], 
        text_target=examples["completion"], 
        truncation=True, 
        padding="max_length", 
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["prompt", "completion"])
# Split into train and test sets
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.2, seed=42)

# Further split the train set into train and validation
train_val_split = split_dataset["train"].train_test_split(test_size=0.1, seed=42)

# Final splits
train_dataset = train_val_split["train"]
validation_dataset = train_val_split["test"]
test_dataset = split_dataset["test"]






Map: 100%|██████████| 4671/4671 [00:00<00:00, 14597.82 examples/s]


In [1]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

GPU memory occupied: 1114 MB.


In [14]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    # auto_find_batch_size=True,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True,
    eval_accumulation_steps=1
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_50429/4290677459.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/10086 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 100.50 MiB is free. Including non-PyTorch memory, this process has 6.81 GiB memory in use. Of the allocated memory 6.49 GiB is allocated by PyTorch, and 130.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)